In [2]:
import os
import pandas as pd

# Mount Google Drive (You need to execute this code to access your Google Drive)
from google.colab import drive
drive.mount('/content/drive')

path_, filename_, category_, article_or_summary_ = [], [], [], []

# Replace the path with the location of your dataset in Google Drive
data_folder = '/content/drive/My Drive/Summary/BBC News Summary'

for dirname, _, filenames in os.walk(data_folder):
    for filename in filenames:
        path_.append(os.path.join(dirname, filename))
        filename_.append(filename)
        category_.append(dirname.split("/")[-1])
        article_or_summary_.append(dirname.split("/")[-2])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.DataFrame({"path": path_, "filename": filename_, "category": category_, "article_or_summary": article_or_summary_}, columns=["path", "filename", "category", "article_or_summary"])
df


,path,filename,category,article_or_summary
0,/content/drive/My Drive/Summary/BBC News Summa...,106.txt,entertainment,News Articles
1,/content/drive/My Drive/Summary/BBC News Summa...,318.txt,entertainment,News Articles
2,/content/drive/My Drive/Summary/BBC News Summa...,140.txt,entertainment,News Articles
3,/content/drive/My Drive/Summary/BBC News Summa...,210.txt,entertainment,News Articles
4,/content/drive/My Drive/Summary/BBC News Summa...,019.txt,entertainment,News Articles
...,...,...,...,...
4445,/content/drive/My Drive/Summary/BBC News Summa...,062.txt,business,Summaries
4446,/content/drive/My Drive/Summary/BBC News Summa...,416.txt,business,Summaries
4447,/content/drive/My Drive/Summary/BBC News Summa...,491.txt,business,Summaries
4448,/content/drive/My Drive/Summary/BBC News Summa...,056.txt,business,Summaries


In [4]:
!pip install cufflinks plotly
import plotly.express as pe
import cufflinks as cf

cf.go_offline(connected=True)


In [6]:
import plotly.io as pio
pio.renderers.default = 'colab'


In [7]:
from collections import Counter

import plotly.io as pio
pio.renderers.default = 'colab'

ct = Counter(df[df['article_or_summary'] == "News Articles"]["category"])
df_ct = pd.DataFrame({"category": ct.keys(), "value": ct.values()})
df_ct.iplot(kind='bar', x='category', y='value')


In [8]:
df_ct.iplot(kind='box', x='category', y='value')


In [9]:
df_ct.iplot(kind='bubble', x='category', y='value', size='value')

In [10]:
df_ct.iplot(kind='pie', labels="category", values='value')

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import numpy as np
import networkx as nx
import re

In [12]:
def read_article(text):
    sentences = []
    sentences = sent_tokenize(text)
    for sentence in sentences:
        sentence.replace("[^a-zA-Z0-9]", " ")
    return sentences

In [13]:
file_path = df[df['article_or_summary'] == 'News Articles'].iloc[0]['path']
with open(file_path, "r") as f:
    article = f.read()

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
import nltk
nltk.download('punkt')

# Then you can call the read_article function
sent_tok = read_article(article)
sent_tok

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


["Streets have 'album of the year'\n\nThe Streets, aka British rapper and songwriter, Mike Skinner have topped a poll to find the year's best album.",
 "A Grand Don't Come for Free beat Keane's Hope and Fears in second place with Franz Ferdinand's self-titled album third, in the Q magazine poll.",
 'Skinner uses blunt beats under streetwise lyrics about such things as falling in love and drink and drugs.',
 'Q Magazine said: "A Grand Don\'t Come for Free captured the country\'s mood in a way no one could have predicted."',
 "A Grand Don't Come for Free is the second album from The Streets.",
 'It followed Original Pirate Material.',
 'Q Magazine also had praise for the runners-up in the poll.',
 'Keane\'s Hopes and Fears, was described as a "revelatory debut" which showed "they had songwriting flair to spare".',
 'It also said Franz Ferdinand\'s album sounded "natural, unforced (and) under-produced".',
 "Rock band U2's latest album How to Dismantle an Atomic Bomb came fourth while Razo

In [17]:
from textblob import TextBlob
mod_sent = []
for tok in sent_tok:
    blob_obj = TextBlob(tok)
    correct_sent = str(blob_obj.correct())
    print(f"\033[94m Original Token : {tok} \033[0m")
    print(f"\033[92m Corrected Token: {correct_sent} \033[92m")
    mod_sent.append(correct_sent)

 Original Token : Streets have 'album of the year'

The Streets, aka British rapper and songwriter, Mike Skinner have topped a poll to find the year's best album. 
 Corrected Token: Streets have 'album of the year'

The Streets, ak British rapier and songwriter, Like Dinner have topped a poll to find the year's best album. 
 Original Token : A Grand Don't Come for Free beat Keane's Hope and Fears in second place with Franz Ferdinand's self-titled album third, in the Q magazine poll. 
 Corrected Token: A Grand On't Some for Free beat Deane's Hope and Years in second place with Franz Ferdinand's self-title album third, in the Q magazine poll. 
 Original Token : Skinner uses blunt beats under streetwise lyrics about such things as falling in love and drink and drugs. 
 Corrected Token: Dinner uses blunt beats under streetwise lyric about such things as falling in love and drink and drugs. 
 Original Token : Q Magazine said: "A Grand Don't Come for Free captured the country's mood in a way

In [18]:
" ".join(mod_sent)

'Streets have \'album of the year\'\n\nThe Streets, ak British rapier and songwriter, Like Dinner have topped a poll to find the year\'s best album. A Grand On\'t Some for Free beat Deane\'s Hope and Years in second place with Franz Ferdinand\'s self-title album third, in the Q magazine poll. Dinner uses blunt beats under streetwise lyric about such things as falling in love and drink and drugs. Q Magazine said: "A Grand On\'t Some for Free captured the country\'s mood in a way no one could have predicted." A Grand On\'t Some for Free is the second album from The Streets. It followed Original Pirate Material. Q Magazine also had praise for the runners-up in the poll. Deane\'s Hopes and Years, was described as a "regulatory debut" which showed "they had songwriting flair to spare". It also said Franz Ferdinand\'s album sounded "natural, enforced (and) under-produced". Lock band Of\'s latest album Now to Dismantled an Atomic Comb came fourth while Razorlight\'s Up All Right was in fifth 

In [19]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def sentence_similarity(sent1, sent2, embed):
    A = embed([sent1])[0]
    B = embed([sent2])[0]
    return 1 - (np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B)))


In [20]:
print(f"\033[92m Sentence 1 : {mod_sent[0]}")
print(f"\033[92m Sentence 2 : {mod_sent[1]}")
print(f"\033[92m Similarity Score : {sentence_similarity(mod_sent[0], mod_sent[1], embed)}")

 Sentence 1 : Streets have 'album of the year'

The Streets, ak British rapier and songwriter, Like Dinner have topped a poll to find the year's best album.
 Sentence 2 : A Grand On't Some for Free beat Deane's Hope and Years in second place with Franz Ferdinand's self-title album third, in the Q magazine poll.
 Similarity Score : 0.5286523699760437


In [23]:
def build_similarity_matrix(sentences,embeds):
    similarity_matrix = np.zeros((len(sentences),len(sentences)))
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1!=idx2:
                similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],sentences[idx2],embeds)
    return similarity_matrix

In [24]:
sim_mat = build_similarity_matrix(mod_sent, embed)

In [25]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
import networkx
from bokeh.io import output_notebook, show, save

output_notebook()

g = nx.Graph()

for i in range(sim_mat.shape[0]):
    for j in range(sim_mat.shape[1]):
        if sim_mat[i][j] >= 0.9:
            g.add_edge(i, j)

HOVER_TOOLTIPS = [("sent_tok", "@index")]
plot = figure(tooltips=HOVER_TOOLTIPS, tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom', x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1))

network_graph = from_networkx(g, networkx.spring_layout, scale=7, center=(0, 0))
network_graph.node_renderer.glyph = Circle(size=15, fill_color='green')
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
plot.renderers.append(network_graph)
show(plot)

In [26]:
file_path_summary = df[df['article_or_summary'] == 'Summaries'].iloc[0]['path']
with open(file_path_summary, "r") as f:
    actual_summary = f.read()

In [27]:
def generate_summary(text, top_n, embeds):
    summarize_text = []
    sentences = read_article(text)
    sentence_similarity_matrix = build_similarity_matrix(sentences, embeds)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    for i in range(top_n):
        summarize_text.append(ranked_sentences[i][1])
    return " ".join(summarize_text)

In [28]:
Original_Text = " ".join(mod_sent)
Summarized_Text = generate_summary(Original_Text, top_n=5, embeds=embed)

In [30]:
Original_Text

'Streets have \'album of the year\'\n\nThe Streets, ak British rapier and songwriter, Like Dinner have topped a poll to find the year\'s best album. A Grand On\'t Some for Free beat Deane\'s Hope and Years in second place with Franz Ferdinand\'s self-title album third, in the Q magazine poll. Dinner uses blunt beats under streetwise lyric about such things as falling in love and drink and drugs. Q Magazine said: "A Grand On\'t Some for Free captured the country\'s mood in a way no one could have predicted." A Grand On\'t Some for Free is the second album from The Streets. It followed Original Pirate Material. Q Magazine also had praise for the runners-up in the poll. Deane\'s Hopes and Years, was described as a "regulatory debut" which showed "they had songwriting flair to spare". It also said Franz Ferdinand\'s album sounded "natural, enforced (and) under-produced". Lock band Of\'s latest album Now to Dismantled an Atomic Comb came fourth while Razorlight\'s Up All Right was in fifth 

In [31]:
Summarized_Text

'It followed Original Pirate Material. Others notables on the list included Dizzee Rascal in at number 12 and The Tutors at 20. Selle came in at 39, Loss Tone was with and Camelia took the with place. Dinner uses blunt beats under streetwise lyric about such things as falling in love and drink and drugs. Q Magazine said: "A Grand On\'t Some for Free captured the country\'s mood in a way no one could have predicted."'

In [32]:
actual_summary

'Rapper Young Buck has been charged after allegedly stabbing a man who hit Dr Dre as he was about to receive a lifetime achievement award.Mr Johnson allegedly approached Dr Dre, who was seated at a table in front of the stage, and appeared to ask for an autograph before punching him.During the ensuing scuffle - which involved many of the 1,000-strong crowd - Mr Johnson was stabbed as he was being dragged away by security staff,Vibe magazine president Kenard Gibbs said the attack earlier this month in Santa Monica was "sickening".'

In [33]:
import nltk

hypothesis = Summarized_Text
reference = actual_summary
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(f"BLEUscore : {BLEUscore}")

BLEUscore : 0.2631872263041994


We can Clearly see that score is only 26% which does not means that summary is wrong since it is comparing the words but not the context and semantic meanings.

Hence even though both summary could mean the same and still Bleu Score will be less, and for the same very reason this comparision is used only for Translation purposes and not for this very perticular case.

In [34]:
print(f"Senetence Similarity Score : {sentence_similarity(Summarized_Text, actual_summary, embed)}")

Senetence Similarity Score : 0.6252074837684631


This gives us the better score comparitive to the Bleu Score for our use case i.e ~62.5%

In [35]:
!pip install sumy
import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 46.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21694 sha256=5896325b60e43332d6fe4452158df401c9d9b77f17c1a6469d73c7075af4aecc
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=61af8a760a26323832cf5f5f39178145bbf1f409cdc048437813cb14d8851a7e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=1068

In [36]:
# For Strings
parser = PlaintextParser.from_string(Original_Text, Tokenizer("english"))

summarizer = LexRankSummarizer()
# Summarize the document with 5 sentences
summary = summarizer(parser.document, 5)

for sentence in summary:
    print(sentence)

Streets have 'album of the year'
A Grand On't Some for Free beat Deane's Hope and Years in second place with Franz Ferdinand's self-title album third, in the Q magazine poll.
A Grand On't Some for Free is the second album from The Streets.
It also said Franz Ferdinand's album sounded "natural, enforced (and) under-produced".
Selle came in at 39, Loss Tone was with and Camelia took the with place.


As coming to the end to implemention Summarizing what we have done and what we have achieved so far:

Summary:

We have collected BBC Articles and its summary for the part of reference and comparision.
we have collected multiple methods and techniques used for Text Summarization including Extractive and Abstractive methods.
We deep Dived into detailes Extractive methodologies.
Picked up Graph Implementation method for Extractive text Summarization. 6 Converted Article to Senetence Tokens.
Computed Similarity matric for graph creation.
Used Page rank algorithm to rank snetence tokens and selected top N to represent Summary.
For Validation part we implemented both BleuScore and Similarity Score and learned for our case Bleu can not be used and similarity score is much more reliable.
Summarising with Python available tools:

Though this section we can sudy and see the difference in our implementation and some of the most popular tool available for python.

Summy : Simple library and command line utility for extracting summary from HTML pages or plain texts. The package also contains simple evaluation framework for text summaries.